In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/data/datasets/realestate.csv')

In [7]:
df

,PID,ST_NUM,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,Y,3,1,1000
1,100002000.0,197.0,N,3,1.5,--
2,100003000.0,NaN,N,NaN,1,850
3,100004000.0,201.0,12,1,NaN,700
4,NaN,203.0,Y,3,2,1600
5,100006000.0,207.0,Y,NaN,1,800
6,100007000.0,NaN,NaN,2,HURLEY,950
7,100008000.0,213.0,Y,1,1,NaN
8,100009000.0,215.0,Y,na,2,1800


In [6]:
df.drop(columns='ST_NAME', inplace=True)

# Data Inspection

In a dataframe there are several functions you can use to inspect what the data looks like. We could start with the `head` method to view the columns and guess what the meaning and data type should be. 

In [3]:
df.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600


It can be useful to check the datatypes of the columns. If a column only contains numeric and missin values it will be a float. If you expect a float and it says object, there may be a corrupt value.

In [4]:
df.dtypes

PID             float64
ST_NUM          float64
ST_NAME          object
OWN_OCCUPIED     object
NUM_BEDROOMS     object
NUM_BATH         object
SQ_FT            object
dtype: object

Because often the table is too big to view all the rows, we can use the `unique` method on categorical data to view the different values. And we see for example that in NUM_BEDROOMS in which we expect a number, there are also entries `nan` which means a mssing value but also the text `na` which is not properly recognized as a missing value. Similarly in SQ_FT there is a `--` entry. We can reload the data adding several tokens for missing values.

In [5]:
df.NUM_BEDROOMS.unique()

array(['3', nan, '1', '2', 'na'], dtype=object)

In [6]:
df = pd.read_csv('/data/datasets/realestate.csv', na_values=['n/a', 'na', '--'])

In [7]:
df.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850.0
3,100004000.0,201.0,BERKELEY,12,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0


In [8]:
df.dtypes

PID             float64
ST_NUM          float64
ST_NAME          object
OWN_OCCUPIED     object
NUM_BEDROOMS    float64
NUM_BATH         object
SQ_FT           float64
dtype: object

# Manually replacing values

In OWN_OCCUPIED is a number 12 and this seems to be a mistake. What we can do is replace this with a NaN ourseleves. Note that the datatype of the column is Text.

In NUM_BATH is a value HURLEY which appears to be a mistake. We can replace that with a missing value, and change the datatype of the column to numeric.

In [9]:
df.OWN_OCCUPIED.unique()

array(['Y', 'N', '12', nan], dtype=object)

In [10]:
df.OWN_OCCUPIED.replace('12', np.nan, inplace=True)

In [11]:
df.NUM_BATH.unique()

array(['1', '1.5', nan, '2', 'HURLEY'], dtype=object)

In [12]:
df['NUM_BATH'] = pd.to_numeric(df.NUM_BATH.replace('HURLEY', np.nan))

We can show only rows that contains missing values, because these are the rows we have to make a decision on.

In [13]:
df[df.isnull().any(axis=1)]

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1.0,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2.0,1600.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1.0,800.0
6,100007000.0,NaN,WASHINGTON,NaN,2.0,NaN,950.0
7,100008000.0,213.0,TREMONT,Y,1.0,1.0,NaN
8,100009000.0,215.0,TREMONT,Y,NaN,2.0,1800.0


# Replacing missing values

We can count the number of missing values in each column. In fact, `isnull` returns a boolean index, and in this index True=1 and False=0 therefore sum counts the number of True values.

In this case, PID and ST_NUM do not seem to be features, so no need to replace that. 

In [14]:
df.isnull().sum()

PID             1
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    2
NUM_BEDROOMS    3
NUM_BATH        2
SQ_FT           2
dtype: int64

We can also fill in values with average values if we do not want to loose too many datapoints. There is something to be said for this approach. Alternatively, we could also try to infer the NUM_BEDROOMS from other features such as SQ_FT and use that. This is a more advanced approach which we shall not cover here.

In [15]:
df.NUM_BEDROOMS.fillna(df.NUM_BEDROOMS.mean(), inplace=True)

In [16]:
df.SQ_FT.fillna(df.SQ_FT.mean(), inplace=True)

In [17]:
df

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.000000,1.0,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.000000,1.5,1100.0
2,100003000.0,NaN,LEXINGTON,N,2.166667,1.0,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.000000,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.000000,2.0,1600.0
5,100006000.0,207.0,BERKELEY,Y,2.166667,1.0,800.0
6,100007000.0,NaN,WASHINGTON,NaN,2.000000,NaN,950.0
7,100008000.0,213.0,TREMONT,Y,1.000000,1.0,1100.0
8,100009000.0,215.0,TREMONT,Y,2.166667,2.0,1800.0
